#  Matching Users: this is a test

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
import datetime as dt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from Utils import TransantiagoConstants

import MatchingUsers
import EvasionBuilder

In [2]:
[first_q, second_q, third_q] = EvasionBuilder.loadSinglesEvasion()

complete_evasion = EvasionBuilder.processSinglesEvasiondAndConcat(first_q,second_q,third_q)

complete_evasion_w_codes = EvasionBuilder.mergeTransantiagoCodes(complete_evasion)

processed_evasion = EvasionBuilder.processCompleteEvasionDataFrame(complete_evasion_w_codes)

clean_processed_evasion = EvasionBuilder.deleteDuplicatedInCompleteEvasion(processed_evasion)

The only non-matched user_code services are: 
D06
Number of duplicated rows in complete evasion database is: 6362
Number of collapsed-duplicated rows in complete evasion database is: 3175
Number of rows in complete evasion database without duplicated rows at all is: 93060
Final number of rows in complete evasion database with collapsed duplicated rows is: 96235


In [3]:
clean_processed_evasion.columns.tolist()

['FECHA',
 'HORA',
 'HORA_INICIO',
 'INGRESAN',
 'LUGAR_INICIO',
 'MINUTO',
 'NO_VALIDAN',
 'N_PUERTA',
 'PATENTE',
 'PUERTAS',
 'SERVICIO',
 'SERVICIO_USUARIO',
 'TIEMPO',
 'TP',
 'UN']

In [4]:
clean_processed_evasion.head()

,FECHA,HORA,HORA_INICIO,INGRESAN,LUGAR_INICIO,MINUTO,NO_VALIDAN,N_PUERTA,PATENTE,PUERTAS,SERVICIO,SERVICIO_USUARIO,TIEMPO,TP,UN
0,2017-01-10,7,07:10:00,15,AV. ADOLFO LOPEZ MATEO,10,1,1,BJFH98,3,214,214,7:10:00,P,2.0
1,2017-01-10,7,07:10:00,9,AV. ADOLFO LOPEZ MATEO,12,0,1,BJFH98,3,214,214,7:12:00,P,2.0
2,2017-01-10,7,07:10:00,6,AV. ADOLFO LOPEZ MATEO,13,2,1,BJFH98,3,214,214,7:13:00,P,2.0
3,2017-01-10,7,07:10:00,8,AV. ADOLFO LOPEZ MATEO,18,1,1,BJFH98,3,214,214,7:18:00,P,2.0
4,2017-01-10,7,07:10:00,5,AV. ADOLFO LOPEZ MATEO,20,1,1,BJFH98,3,214,214,7:20:00,P,2.0


In [5]:
print(type(clean_processed_evasion.loc[0,'TIEMPO'])) #should be a string
print(type(clean_processed_evasion.loc[0,'FECHA'])) #should be a timestamp

<class 'str'>
<class 'pandas._libs.tslib.Timestamp'>


* Processing evasion-ddbb before merging

In [6]:
evasion_paradero = clean_processed_evasion[clean_processed_evasion['TP']=='P']
evasion_paradero_first = evasion_paradero[evasion_paradero['N_PUERTA']==1]
print(type(clean_processed_evasion['SERVICIO'][0])) #should be a string

<class 'str'>


In [7]:
print('Number of rows in complete evasion database is: ' + str(len(clean_processed_evasion.index)))
print('Number of rows in evasion in paradero database is: ' + str(len(evasion_paradero.index)))
print('Number of rows in evasion in paradero by first door database is: ' + str(len(evasion_paradero_first.index)))

Number of rows in complete evasion database is: 96235
Number of rows in evasion in paradero database is: 94531
Number of rows in evasion in paradero by first door database is: 33755


* Filtering dates not in common dates

In [8]:
common_dates = TransantiagoConstants.common_dates
common_dates_timestamp = [pd.to_datetime(x) for x in common_dates]
common_dates_evasion = evasion_paradero_first[evasion_paradero_first['FECHA'].isin(common_dates_timestamp)]
len(common_dates_evasion.index) #Remember to store the number.

14756

* <font color='red'>Filtering for one specific common date and analyze it to refactor and debug <- this part should be omitted in the final script. Above methodology is general</font>

In [9]:
example = common_dates_evasion[common_dates_evasion['FECHA']==pd.to_datetime('2017-04-11')]

In [10]:
len(example.index)

1106

In [11]:
from RunSilentlyDailyEtapasBuilder import RunSilentlyDailyEtapasBuilderClass

In [12]:
date = '2017-04-11'
etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
processed_sorted_df = etapas_builder.runProcessedProcess()

Not found in turnstile database: 1829611


In [13]:
len(processed_sorted_df.index)

2598131

In [14]:
#processed_sorted_df = processed_sorted_df[processed_sorted_df['diferencia_tiempo_secs']<=20]

* Processing sorted_df before reducing complexity

In [15]:
processed_sorted_df['servicio_subida'] =  processed_sorted_df['servicio_subida'].str.replace('T','')
processed_sorted_df['servicio_subida'] =  processed_sorted_df['servicio_subida'].str.split(' ').str[0]

In [16]:
processed_sorted_df['sitio_subida'] = processed_sorted_df['sitio_subida'].str.replace("-", "")
processed_sorted_df['sitio_subida'] = processed_sorted_df['sitio_subida'].str.replace(" ", "")

* First: filter etapas by patentes and servicios surveyed in evasion ddbb so to reduce search space

In [17]:
patentes = example['PATENTE'].unique()
patentes = patentes.tolist()
servicios = example['SERVICIO'].unique()
servicios = servicios.tolist()

In [18]:
clean_sorted_df = processed_sorted_df[(processed_sorted_df['sitio_subida'].isin(patentes))&(processed_sorted_df['servicio_subida'].isin(servicios))]

In [19]:
clean_sorted_df = clean_sorted_df.reset_index(drop=True)

In [20]:
len(clean_sorted_df.index)

11937

* New methodology: appending evasion to trx

In [21]:
past_servicio = ''
past_patente = ''
past_time = pd.to_datetime('')
id_exp = 1
clean_sorted_df['idExpedicion'] = ''

for index,row in clean_sorted_df.iterrows():
    actual_servicio = row['servicio_subida']
    actual_patente = row['sitio_subida']
    actual_time = row['t_subida']
    
    #SOMETHING
    if((past_servicio==actual_servicio)&(actual_time - past_time <= (pd.to_datetime('2017-04-11 00:15:00')-pd.to_datetime('2017-04-11 00:00:00')))):
        clean_sorted_df.loc[index,'idExpedicion'] = id_exp

    elif((past_servicio==actual_servicio)&(actual_time - past_time > (pd.to_datetime('2017-04-11 00:15:00')-pd.to_datetime('2017-04-11 00:00:00')))):
        id_exp = id_exp + 1
        clean_sorted_df.loc[index,'idExpedicion'] = id_exp
        
    elif((past_servicio!=actual_servicio)):
        id_exp = 1
        clean_sorted_df.loc[index,'idExpedicion'] = id_exp
        

    past_servicio = actual_servicio
    past_patente = actual_patente
    past_time = actual_time
    

In [22]:
clean_sorted_df.head()

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete,idExpedicion
0,2381078,3854261808,1,BUS,2017-04-11 08:00:54,507,L-10-55-5-PO,BJFB62,NaT,False,False,False,NaT,NaN,False,False,1
1,2386934,3857420299,1,BUS,2017-04-11 08:02:49,507,L-10-82-5-PO,BJFB62,NaT,False,True,True,NaT,NaN,False,False,1
2,856791,1182024974,1,BUS,2017-04-11 08:03:46,507,L-10-37-20-NS,BJFB62,NaT,False,True,True,NaT,NaN,False,False,1
3,2207200,3562465182,2,BUS,2017-04-11 08:03:48,507,L-10-37-20-NS,BJFB62,NaT,True,True,True,00:00:02,2.0,False,False,1
4,646070,679843243,1,BUS,2017-04-11 08:06:16,507,L-10-21-45-NS,BJFB62,NaT,False,True,True,NaT,NaN,False,False,1


In [23]:
f = {'t_subida':['min', 'max', 'count'], 'diferencia_tiempo_secs':['mean']}
grouped_clean_sorted_df = clean_sorted_df.groupby(['sitio_subida','servicio_subida','idExpedicion','par_subida']).agg(f)
grouped_clean_sorted_df = grouped_clean_sorted_df.reset_index()
grouped_clean_sorted_df.columns = [''.join(col).strip() for col in grouped_clean_sorted_df.columns.values]
grouped_clean_sorted_df = grouped_clean_sorted_df.sort_values(by=['sitio_subida', 'servicio_subida', 'idExpedicion', 't_subidamin'], ascending=[True, True, True, True])
grouped_clean_sorted_df = grouped_clean_sorted_df.reset_index(drop=True)

In [24]:
i = 0
past_plate = '-'
past_service = '-'

past_par_subida = ['']

past_initial_hour = pd.to_datetime('2017-04-11 00:00:00')
past_end_hour = pd.to_datetime('2017-04-11 00:00:00')
past_id_expedicion = '-'

grouped_clean_sorted_df['start_cut']=''
grouped_clean_sorted_df['end_cut']=''

from IPython.core.debugger import Tracer

for index,row in grouped_clean_sorted_df.iterrows():
    
    future_index = index+1
    
    actual_plate = row['sitio_subida']
    actual_service = row['servicio_subida']
    actual_initial_hour = row['t_subidamin']
    actual_end_hour = row['t_subidamax']
        
    actual_id_expedicion = row['idExpedicion']
    
    if ((actual_plate!=past_plate)|(actual_service!=past_service)|(actual_id_expedicion!=past_id_expedicion)): #Assuming a pre-defined value, i.e., 30 seconds.
        start_cut = actual_initial_hour - (pd.to_datetime('2017-04-11 00:01:00') - pd.to_datetime('2017-04-11 00:00:30'))
    else:
        start_cut = actual_initial_hour - (actual_initial_hour - past_end_hour)/2
    
    
    if future_index >= len(grouped_clean_sorted_df.index): #We are at the end of the ddff. Assuming a pre-defined value, i.e., 30 seconds 
        end_cut = actual_end_hour + (pd.to_datetime('2017-04-11 00:01:00') - pd.to_datetime('2017-04-11 00:00:30'))
        
    else:        
        future_plate = grouped_clean_sorted_df.loc[future_index,'sitio_subida']
        future_service = grouped_clean_sorted_df.loc[future_index,'servicio_subida']
        future_initial_hour = grouped_clean_sorted_df.loc[future_index,'t_subidamin']
        future_end_hour = grouped_clean_sorted_df.loc[future_index,'t_subidamax']
        future_id_expedicion = grouped_clean_sorted_df.loc[future_index,'idExpedicion']
        
        if ((actual_plate!=future_plate)|(actual_service!=future_service)|(actual_id_expedicion!=future_id_expedicion)): #Changing service or plate... assuming a pre-defined value, i.e., 30 seconds
            end_cut = actual_end_hour + (pd.to_datetime('2017-04-11 00:01:00') - pd.to_datetime('2017-04-11 00:00:30'))
        else:
            end_cut = actual_end_hour + (future_initial_hour - actual_end_hour)/2
   
    past_initial_hour = actual_initial_hour
    past_end_hour = actual_end_hour
    past_id_expedicion = actual_id_expedicion
    past_plate = actual_plate
    past_service = actual_service
    grouped_clean_sorted_df.loc[index,'start_cut']=start_cut
    grouped_clean_sorted_df.loc[index,'end_cut']=end_cut

In [25]:
example.loc[:,'TIEMPO'] = example.loc[:,'FECHA'].dt.strftime('%Y-%m-%d') + ' ' + example.loc[:,'TIEMPO']
example['TIEMPO'] = pd.to_datetime(example['TIEMPO'])

C:\Users\leoca_000\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\leoca_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
example = example.reset_index(drop =True)

In [27]:
for index,row in grouped_clean_sorted_df.iterrows():
    actual_plate = row['sitio_subida']
    actual_service = row['servicio_subida']
    start_cut = row['start_cut']
    end_cut = row['end_cut']
    
    actual_util_df = example[(example['PATENTE']==actual_plate)&(example['SERVICIO']==actual_service)&(start_cut<=example['TIEMPO'])&(example['TIEMPO']<=end_cut)]
    actual_util_df = actual_util_df[(actual_util_df['INGRESAN'] > actual_util_df['NO_VALIDAN'])] #Be aware of this condition
    actual_ev_obs = len(actual_util_df.index) #Mantains order of appearence
        
    grouped_clean_sorted_df.loc[index,'count_ev_obs'] = actual_ev_obs
    grouped_clean_sorted_df.loc[index,'ingresan'] = actual_util_df['INGRESAN'].sum()
    grouped_clean_sorted_df.loc[index,'no_validan'] = actual_util_df['NO_VALIDAN'].sum()
#    grouped_clean_sorted_df.loc[index,'tiempo_ev'] = actual_util_df['TIEMPO']

In [28]:
grouped_clean_sorted_df.head()

,sitio_subida,servicio_subida,idExpedicion,par_subida,t_subidamin,t_subidamax,t_subidacount,diferencia_tiempo_secsmean,start_cut,end_cut,count_ev_obs,ingresan,no_validan
0,BJFB62,507,1,L-10-55-5-PO,2017-04-11 08:00:54,2017-04-11 08:00:54,1,NaN,2017-04-11 08:00:24,2017-04-11 08:01:51.500000,0.0,0.0,0.0
1,BJFB62,507,1,L-10-82-5-PO,2017-04-11 08:02:49,2017-04-11 08:02:49,1,NaN,2017-04-11 08:01:51.500000,2017-04-11 08:03:17.500000,0.0,0.0,0.0
2,BJFB62,507,1,L-10-37-20-NS,2017-04-11 08:03:46,2017-04-11 08:03:48,2,2.0,2017-04-11 08:03:17.500000,2017-04-11 08:05:02,0.0,0.0,0.0
3,BJFB62,507,1,L-10-21-45-NS,2017-04-11 08:06:16,2017-04-11 08:06:16,1,NaN,2017-04-11 08:05:02,2017-04-11 08:10:43.500000,0.0,0.0,0.0
4,BJFB62,507,1,T-10-455-SN-5,2017-04-11 08:15:11,2017-04-11 08:15:11,1,NaN,2017-04-11 08:10:43.500000,2017-04-11 08:15:46.500000,0.0,0.0,0.0


In [29]:
len(grouped_clean_sorted_df.index)

3994

In [ ]:
processed_sorted_df[(processed_sorted_df['sitio_subida']=='BJFB62')&(processed_sorted_df['servicio_subida']=='507')&(processed_sorted_df['par_subida']=='T-18-156-OP-40')]

In [ ]:
grouped_clean_sorted_df['evasion_rate'] = grouped_clean_sorted_df['no_validan']/grouped_clean_sorted_df['ingresan']

In [ ]:
grouped_clean_sorted_df_test = grouped_clean_sorted_df[grouped_clean_sorted_df['diferencia_tiempo_secsmean'].notnull()]

In [ ]:
grouped_clean_sorted_df_test_clean = grouped_clean_sorted_df_test[grouped_clean_sorted_df_test['evasion_rate'].notnull()]

In [ ]:
grouped_clean_sorted_df_test_clean.head()

In [ ]:
torniquetesDataPath = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/03_BUSESTORNIQUETE/Avance_Consolidado_v2.xlsx"
busesTorniquete_df = pd.read_excel(torniquetesDataPath)
busesTorniquete_df.columns=['sitio_subida','fecha_instalacion']

busesTorniquete_df['sitio_subida'] = busesTorniquete_df['sitio_subida'].str.replace("-", "")
busesTorniquete_df['sitio_subida'] = busesTorniquete_df['sitio_subida'].str.replace(" ", "")

In [ ]:
busesTorniquete_df.head()

In [ ]:
merged_turnstiles_df = pd.merge(grouped_clean_sorted_df_test_clean,busesTorniquete_df, on='sitio_subida', how='left')
checking_missing = pd.isnull(merged_turnstiles_df['fecha_instalacion'])
print('Not found in turnstile database: ' + str(sum(checking_missing))) #Without turnstiles, then NaT.
merged_turnstiles_df['fecha_instalacion'] = pd.to_datetime(merged_turnstiles_df.fecha_instalacion)

In [ ]:
grouped_clean_sorted_df_test_clean_noturns = merged_turnstiles_df[merged_turnstiles_df['fecha_instalacion'].isnull()]

In [ ]:
grouped_clean_sorted_df_test_clean_noturns.head()

In [ ]:
fig_1, ax_1 = plt.subplots()

ax_1.scatter(grouped_clean_sorted_df_test_clean_noturns['diferencia_tiempo_secsmean'],grouped_clean_sorted_df_test_clean_noturns['evasion_rate'])
#ax_1.set_title('Evasion vs. Mean Time - No Turns, By Bus and Service, Original')
#ax_1.set_xlabel('Mean Time Interval [s]')
#ax_1.set_ylabel('Evasion Rate [%]')